In [71]:
import pandas as pd
import numpy as np
import json
import requests
import openpyxl
from bs4 import BeautifulSoup

# SLEEP QUALITY AND DURATION DATA

In [8]:
# Importing the two datasets
country_abbrevs = pd.read_csv('data/country_abbrev_list.csv')

re = requests.get('https://sandman.sleepcycle.com/data/per-country-stats')
country_sleep_data = pd.DataFrame(re.json())

In [9]:
country_sleep_data

,country,avg_sleep_quality,avg_duration,avg_snore_duration,avg_bedtime,avg_wakeup,group_size
0,ca,0.7706280786149319,27203.393388256663,2658.828667280861,-0.010468966378401922,0.30500672811254403,28646
1,co,0.7297271999818904,25615.27579824156,1437.7857592261742,-0.03228318759677625,0.2644591619912697,2161
2,in,0.7060974741852611,25093.229276895945,2137.2825717013548,0.004082686733507055,0.2950252965085311,1134
3,il,0.7406121077804889,26042.273445212242,2051.836718594944,0.017049587070040857,0.31956485420204445,2026
4,se,0.786575637584524,27685.027529392293,3116.2711398811175,-0.007667250763352499,0.3134636193388775,24156
...,...,...,...,...,...,...,...
57,cy,0.7502196809844399,27078.2886002886,2816.03259211972,0.0043422235917116225,0.31831668729306556,693
58,jp,0.6784701596828577,24236.974481437002,3323.9126377116636,0.00410389190726472,0.2842500281637383,100711
59,gb,0.7840466475722139,27963.86005062065,2777.4097512704925,-0.00660779543906787,0.31820536414364314,65981
60,pe,0.7117202666169559,25158.92667509482,1714.1829438347725,-0.010349246817979626,0.2795599722538905,791


In [10]:
# Merging the two datasets
country_sleep_data['country'] = country_sleep_data['country'].str.upper()
sleep_by_country = pd.merge(country_sleep_data, country_abbrevs, left_on = 'country', right_on = 'Code').drop(['country', 'Code'], axis=1).set_index('Name')

In [11]:
# Converting all the strings to floats and converting sleep values to hours or minutes (from seconds)
sleep_by_country = sleep_by_country.astype(float)
sleep_by_country['avg_duration'] = round(sleep_by_country['avg_duration'] / 3600, 2)
sleep_by_country['avg_snore_duration'] = round(sleep_by_country['avg_snore_duration'] / 60, 2)

In [12]:
# Renaming columns appropriately and dropping unnecessary columns
sleep_by_country = sleep_by_country.rename(columns = {'avg_sleep_quality': 'Average Sleep Quality (0-1)', 'avg_duration': 'Average Sleep Duration (Hours)', 'avg_snore_duration': 'Average Snore Duration (Minutes)'})
sleep_by_country = sleep_by_country.drop(['avg_bedtime', 'avg_wakeup', 'group_size'], axis=1)

In [13]:
sleep_by_country

,Average Sleep Quality (0-1),Average Sleep Duration (Hours),Average Snore Duration (Minutes)
Name,,,
Canada,0.770628,7.56,44.31
Colombia,0.729727,7.12,23.96
India,0.706097,6.97,35.62
Israel,0.740612,7.23,34.20
Sweden,0.786576,7.69,51.94
...,...,...,...
Cyprus,0.750220,7.52,46.93
Japan,0.678470,6.73,55.40
United Kingdom,0.784047,7.77,46.29


# SOCIOECONOMIC DATA

In [16]:
# Importing and cleaning the dataset (removing some columns, making sure to only keep the most recent year for each country)
socioeconomic_data = pd.read_csv('data/raw_socioeconomic_data.csv', encoding = 'latin1')
socioeconomic_data = socioeconomic_data.drop(['unid', 'wbid', 'popshare', 'yrseduc'], axis=1)
socioeconomic_data = socioeconomic_data[socioeconomic_data['year'] == 2010]
socioeconomic_data = socioeconomic_data.set_index('country')

In [17]:
socioeconomic_data

,year,SES,gdppc
country,,,
Afghanistan,2010,5.676400,1662.8035
Angola,2010,21.247763,6492.1768
Albania,2010,74.860367,9927.1758
United Arab Emirates,2010,89.092285,57406.7380
Argentina,2010,79.750809,18794.2700
...,...,...,...
Vietnam,2010,38.222027,4486.2612
Yemen,2010,19.492294,4481.5547
South Africa,2010,71.227959,12028.9340


# WHO ENVIRONMENTAL DATA

In [54]:
# Importing the dataset
air_df = pd.read_csv('data/who_2022_air_quality.csv', encoding='latin1')

In [55]:
# Cleaning the dataset by getting only most recent year for each country and dropping unnecessary columns
air_df = air_df.loc[air_df.groupby('WHO Country Name')['Measurement Year'].idxmax()].set_index('WHO Country Name')
air_df = air_df.drop(['WHO Region', 'ISO3', 'Version of the database', 'Number and type of monitoring stations', 'City or Locality', 'Reference', 'Status', 'Unnamed: 15', 'Unnamed: 16', 'PM25 temporal coverage (%)', 'PM10 temporal coverage (%)', 'NO2 temporal coverage (%)'], axis=1)

In [56]:
air_df

,Measurement Year,PM2.5 (?g/m3),PM10 (?g/m3),NO2 (?g/m3)
WHO Country Name,,,,
Afghanistan,2019,119.77,NaN,NaN
Albania,2019,10.32,NaN,NaN
Algeria,2019,21.53,NaN,NaN
Andorra,2019,NaN,24.58,31.01
Argentina,2019,NaN,25.50,18.25
...,...,...,...,...
United States of America,2019,NaN,18.30,17.70
Uruguay,2018,12.00,19.00,NaN
Uzbekistan,2019,41.96,NaN,NaN


# New Environmental Dataset

In [79]:
#re = requests.get('https://en.wikipedia.org/wiki/List_of_countries_by_air_pollution')
#soup = BeautifulSoup(re.text, 'html.parser')
#table = soup.find('table', {'class': 'wikitable sortable jquery-tablesorter'})
#df = pd.read_html(str(table))[0]
#print(df.head())

# Merging our 3 Datasets

In [ ]:
merged_df = pd.merge(pd.merge(sleep_by_country, socioeconomic_data, left_index=True, right_index=True, how='left'), air_df, left_index=True, right_index=True, how='left').drop(['year', 'Measurement Year'], axis=1)

In [58]:
merged_df.head()

,Average Sleep Quality (0-1),Average Sleep Duration (Hours),Average Snore Duration (Minutes),SES,gdppc,PM2.5 (?g/m3),PM10 (?g/m3),NO2 (?g/m3)
Canada,0.770628,7.56,44.31,93.772118,40699.3550,7.00,NaN,11.0
Colombia,0.729727,7.12,23.96,59.636009,10900.5040,10.49,NaN,NaN
India,0.706097,6.97,35.62,22.542639,4404.5376,45.00,74.0,25.0
Israel,0.740612,7.23,34.20,86.517677,29599.7930,NaN,63.5,NaN
Sweden,0.786576,7.69,51.94,91.959633,42942.5630,NaN,NaN,1.2


In [61]:
merged_df['Average Sleep Quality (0-1)'].describe()

count    54.000000
mean      0.751335
std       0.028544
min       0.678470
25%       0.732169
50%       0.751839
75%       0.771441
max       0.804231
Name: Average Sleep Quality (0-1), dtype: float64

In [59]:
merged_df['Average Sleep Duration (Hours)'].describe()

count    54.000000
mean      7.405185
std       0.269358
min       6.730000
25%       7.232500
50%       7.425000
75%       7.587500
max       7.910000
Name: Average Sleep Duration (Hours), dtype: float64

In [62]:
df_dict = {}
for col in merged_df.columns:
    df_dict[col] = merged_df[col].describe()

In [66]:
pd.DataFrame(df_dict).drop(['count', '25%', '50%', '75%'])

,Average Sleep Quality (0-1),Average Sleep Duration (Hours),Average Snore Duration (Minutes),SES,gdppc,PM2.5 (?g/m3),PM10 (?g/m3),NO2 (?g/m3)
mean,0.751335,7.405185,43.418148,77.746842,28643.999354,15.258182,35.366053,17.603514
std,0.028544,0.269358,8.744656,16.737152,17980.950306,10.406804,41.174761,8.351899
min,0.678470,6.730000,22.520000,22.542639,4404.537600,4.010000,11.000000,1.200000
max,0.804231,7.910000,63.250000,97.551964,89727.039000,46.350000,249.000000,36.830000
